In [1]:
# https://pieriantraining.com/tensorflow-lstm-example-a-beginners-guide/
# https://www.tensorflow.org/guide/keras/working_with_rnns

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf
import keras
from keras.models import load_model
from keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import joblib

In [3]:
dataset = pd.read_csv('../../data/preprocessed_data.csv',encoding='ISO-8859-1')
dataset = dataset.drop(['Unnamed: 0'], axis=1)
dataset = dataset.dropna(subset=['Text'])
dataset = dataset.sample(frac=0.1, random_state=42)
dataset

,Text,Source,Human
287452,ING AsiaPacific Companys Problems Research Pap...,Human,1
222199,Crisis Love Inquiry Essay Critical Writing fol...,Human,1
453600,Sure sex segregation makes lot sense many spor...,Human,1
276338,Christianity Islam Values Essay Christianity f...,Human,1
78217,Becca liked swim practiced everyday hours ente...,GLM-130B,0
...,...,...,...
663613,Mass Eoghan Chada 10 brother Ruairi 5 said St ...,OPT-30B,0
285976,Asian Teachers Polish Lesson Perfection Stigle...,Human,1
679335,Move knife slowly avoid slipping accidentally ...,OPT-6.7B,0
775773,Good dreams likely occur person feeling relaxe...,Text-Davinci-003,0


In [4]:
# Initialize the tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['Text'])

In [5]:
# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(dataset['Text'])

# Pad sequences to ensure uniform input length
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [6]:
# unitializing label encoder
label_encoder = LabelEncoder()
label_encoder.fit(dataset['Human'])

# labels to numerical format conversion
encoded_labels = label_encoder.transform(dataset['Human'])

In [14]:
# LSTM instantiation
lstm = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    LSTM(64),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [15]:
lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
# training phase
lstm.fit(padded_sequences, encoded_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


1973/1973 ━━━━━━━━━━━━━━━━━━━━ 50s 25ms/step - accuracy: 0.6778 - loss: 0.6038 - val_accuracy: 0.7301 - val_loss: 0.5479
Epoch 2/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 35s 18ms/step - accuracy: 0.7617 - loss: 0.4779 - val_accuracy: 0.8238 - val_loss: 0.3669
Epoch 3/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 33s 17ms/step - accuracy: 0.8132 - loss: 0.3816 - val_accuracy: 0.8063 - val_loss: 0.3961
Epoch 4/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 32s 16ms/step - accuracy: 0.8404 - loss: 0.3367 - val_accuracy: 0.8366 - val_loss: 0.3524
Epoch 5/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 44s 22ms/step - accuracy: 0.8714 - loss: 0.2807 - val_accuracy: 0.8369 - val_loss: 0.3642
Epoch 6/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 90s 26ms/step - accuracy: 0.8910 - loss: 0.2431 - val_accuracy: 0.8385 - val_loss: 0.3689
Epoch 7/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.9102 - loss: 0.2049 - val_accuracy: 0.8411 - val_loss: 0.3949
Epoch 8/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 31s 15ms/step - accuracy: 0.9252 - loss: 0.17

In [17]:
# 0.9539

In [19]:
lstm.save('../../models/NeuralNetworks/lstm_one_95.keras')